In [2]:
#P58 - Transformação da base flags

import pandas as pd

#---------------------------------------------------------------------
#1-Importa a base de dados
#---------------------------------------------------------------------
flags = pd.read_csv('/workspaces/Pandas_python_livro_casa_do_codigo/main_project/flags.csv')

#---------------------------------------------------------------------
#2-Conversão dos atributos do Grupo 2
#   De: Categóricos não binários
#   Para: Categóricos binários
#---------------------------------------------------------------------
for c in flags.columns:
    if c in ['landmass','zone',
             'language','religion','mainhue',
             'topleft','botright']:
        dummies = pd.get_dummies(flags[c], prefix=c)
        flags = flags.join(dummies)


#---------------------------------------------------------------------
#3-Normalização dos atributos dos Grupo 3 e 4
#   De: Numéricos contínuos e discretos
#   Para: Numéricos com valores na faixa entre 0 e 1
#---------------------------------------------------------------------
for c in flags.columns:
    if c in ['area','population',
             'bars','stripes','colours',
             'circles','crosses','saltires',
             'quarters','sunstars']:

        c_max = max(flags[c])
        c_min = min(flags[c])
        flags[c] = (flags[c] - c_min) / (c_max - c_min)


#---------------------------------------------------------------------
#4-Exclusão dos atributos indesejados
#---------------------------------------------------------------------
flags = flags.drop(columns=['name', 'landmass',
                            'zone', 'language',
                            'religion', 'mainhue',
                            'topleft', 'botright',
                           ])


#---------------------------------------------------------------------
#5-Imprime a configuração final de flags
#---------------------------------------------------------------------
#Imprime as primeiras linhas
print('Head():'); print(flags.head())
print('-----------------------------------------')

#Imprime as últimas linhas
print('Tail():'); print(flags.tail())
print('-----------------------------------------')


#---------------------------------------------------------------------
#6-Salva o dataset alterado para um arquivo
#---------------------------------------------------------------------
flags.to_csv('/workspaces/Pandas_python_livro_casa_do_codigo/main_project/flags_transf.csv', sep=";", index=False)

Head():
       area  population  bars   stripes   colours  red  green  blue  gold  \
0  0.028926    0.015873   0.0  0.214286  0.571429    1      1     0     1   
1  0.001295    0.002976   0.0  0.000000  0.285714    1      0     0     1   
2  0.106598    0.019841   0.4  0.000000  0.285714    1      1     0     0   
3  0.000000    0.000000   0.0  0.000000  0.571429    1      0     1     1   
4  0.000000    0.000000   0.6  0.000000  0.285714    1      0     1     1   

   white  ...  topleft_red  topleft_white  botright_black  botright_blue  \
0      1  ...        False          False           False          False   
1      0  ...         True          False           False          False   
2      1  ...        False          False           False          False   
3      1  ...        False          False           False          False   
4      0  ...        False          False           False          False   

   botright_brown  botright_gold  botright_green  botright_orange  \
0  

# Implementação do modelo

In [3]:
#Parte 1 importação das bibliotes
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [4]:
#Parte 2 carrega a base de dados treinamentos (feita no cap 6, transformando as colunas)
flags = pd.read_csv('/workspaces/Pandas_python_livro_casa_do_codigo/main_project/flags_transf.csv',sep= ';')
print(flags.columns)

Index(['area', 'population', 'bars', 'stripes', 'colours', 'red', 'green',
       'blue', 'gold', 'white', 'black', 'orange', 'circles', 'crosses',
       'saltires', 'quarters', 'sunstars', 'crescent', 'triangle', 'icon',
       'animate', 'text', 'landmass_1', 'landmass_2', 'landmass_3',
       'landmass_4', 'landmass_5', 'landmass_6', 'zone_1', 'zone_2', 'zone_3',
       'zone_4', 'language_1', 'language_2', 'language_3', 'language_4',
       'language_5', 'language_6', 'language_7', 'language_8', 'language_9',
       'language_10', 'religion_0', 'religion_1', 'religion_2', 'religion_3',
       'religion_4', 'religion_5', 'religion_6', 'religion_7', 'mainhue_black',
       'mainhue_blue', 'mainhue_brown', 'mainhue_gold', 'mainhue_green',
       'mainhue_orange', 'mainhue_red', 'mainhue_white', 'topleft_black',
       'topleft_blue', 'topleft_gold', 'topleft_green', 'topleft_orange',
       'topleft_red', 'topleft_white', 'botright_black', 'botright_blue',
       'botright_brown', 'b

In [5]:
#Parte 3 configura os parâmetros e variáveis auxiliares
# valor de k a ser usado no k-nn (qtd de vizinhos considerados na classificação)
k = 3

#nomes das labels (rótulos de classe)
labels = ['red','green','blue','gold','white','black','orange']

# numero de labels = 7
n_labels = len(labels)

# numero de registros da base de treinamento = 194
n_registros = flags.shape[0]


In [6]:
#Parte 4 cria um classificador com o algoritmo k-NN 
# e faz a estimativa do desempenho usando o método LOO(Leave-One-Out)

#4.1 faz um loop que percorre cada rótulo

for i in range(0,n_labels):
    print('-----------------------------------------')
    print("PROCESSANDO O RÓTULO ", labels[i])

    #-----------------------------------------------------------
    # 4.1.1 instancia uma matriz de confusão para o rótulo j
    #-----------------------------------------------------------
    mc = pd.DataFrame({'predito_nao':[0,0],
                       'predito_sim':[0,0]},
                       index = ['real_nao','real_sim'])
    #-----------------------------------------------------------
    # 4.1.2 divide a base de treinamento verticalmente
    #       em duas partes: X (atributos preditivos) e
    #                      Y (atributo classe)
    #-----------------------------------------------------------
    X = flags.drop(columns=labels)
    Y = flags[labels[i]]


    #4.2 loop que realiza o LOO para os rótulos em no loop i

    for j in range(0,n_registros):
        
        #-----------------------------------------------------------
        # 4.2.1 Separa os dados que serão utilizados para treinar o modelo
        #-----------------------------------------------------------
        X_treino = X.drop([j])
        Y_treino = Y.drop([j])

        #-------------------------------------------
        # 4.2.2 Separa o objeto de teste
        #-------------------------------------------
        X_teste = X.iloc[[j],:]
        Y_teste = Y.iloc[j]
        #-----------------------------------------------------------
        # 4.2.3 Treinamento do modelo k-NN com os dados
        #       de treino
        #-----------------------------------------------------------
        modelo = KNeighborsClassifier(n_neighbors=k)
        modelo.fit(X_treino,Y_treino)
        
        #-----------------------------------------------------------
        # 4.2.4 Teste do modelo k-NN com o objeto de teste
        #-----------------------------------------------------------
        pred = modelo.predict(X_teste)

        #-----------------------------------------------------------
        # 4.2.5 Atualiza a celula adequada da matriz de confusão em função do resultado do teste
        #-----------------------------------------------------------
        if (Y_teste == 0):
            if (pred == 0): mc.iloc[0,0]+=1
            if (pred == 1): mc.iloc[0,1]+=1
        else:
            if (pred == 0): mc.iloc[1,0]+=1
            if (pred == 1): mc.iloc[1,1]+=1

    #-----------------------------------------------------------
    # 4.3 - Fim do LOO para o rótulo i:
    #       imprime a sua matriz de confusão e acurácia
    #-----------------------------------------------------------
    print(mc)
    acuracia = (mc.iloc[0,0] + mc.iloc[1,1]) / n_registros
    print('Acurácia = ',round(acuracia,2))

-----------------------------------------
PROCESSANDO O RÓTULO  red
          predito_nao  predito_sim
real_nao           20           21
real_sim           22          131
Acurácia =  0.78
-----------------------------------------
PROCESSANDO O RÓTULO  green
          predito_nao  predito_sim
real_nao           88           15
real_sim           17           74
Acurácia =  0.84
-----------------------------------------
PROCESSANDO O RÓTULO  blue
          predito_nao  predito_sim
real_nao           64           31
real_sim           28           71
Acurácia =  0.7
-----------------------------------------
PROCESSANDO O RÓTULO  gold
          predito_nao  predito_sim
real_nao           81           22
real_sim           37           54
Acurácia =  0.7
-----------------------------------------
PROCESSANDO O RÓTULO  white
          predito_nao  predito_sim
real_nao           18           30
real_sim           24          122
Acurácia =  0.72
-----------------------------------------
PROC

# Testando diferentes k para ver o desempenho do modelo
## Vou usar a Medição de Acurácia identificar qual a melhor média de Acurácia

In [21]:
# Lista para armazenar resultados
resultados = []

# Loop para diferentes valores de k
for k in range(3, 21):
    print("\n========================")
    print(f"Testando k = {k}")
    print("========================")
    
    for i in range(0, n_labels):
        print('-----------------------------------------')
        print("PROCESSANDO O RÓTULO ", labels[i])

        mc = pd.DataFrame({'predito_nao':[0,0],
                           'predito_sim':[0,0]},
                           index=['real_nao','real_sim'])

        X = flags.drop(columns=labels)
        Y = flags[labels[i]]

        for j in range(0, n_registros):
            X_treino = X.drop([j])
            Y_treino = Y.drop([j])
            X_teste = X.iloc[[j], :]
            Y_teste = Y.iloc[j]

            modelo = KNeighborsClassifier(n_neighbors=k)
            modelo.fit(X_treino, Y_treino)
            pred = modelo.predict(X_teste)

            if (Y_teste == 0):
                if (pred == 0): mc.iloc[0,0]+=1
                else: mc.iloc[0,1]+=1
            else:
                if (pred == 0): mc.iloc[1,0]+=1
                else: mc.iloc[1,1]+=1

        acuracia = (mc.iloc[0,0] + mc.iloc[1,1]) / n_registros
        print('Acurácia = ', round(acuracia, 2))

        # Salva resultado
        resultados.append({"k": k, "label": labels[i], "acuracia": acuracia})

# ----- Resumo final -----
df_resultados = pd.DataFrame(resultados)
print("\nResumo final:")
print(df_resultados)

# Média por k
media_por_k = df_resultados.groupby("k")["acuracia"].mean().reset_index()
print("\nMédia geral das acurácias por k:")
print(media_por_k)



Testando k = 3
-----------------------------------------
PROCESSANDO O RÓTULO  red
Acurácia =  0.78
-----------------------------------------
PROCESSANDO O RÓTULO  green
Acurácia =  0.84
-----------------------------------------
PROCESSANDO O RÓTULO  blue
Acurácia =  0.7
-----------------------------------------
PROCESSANDO O RÓTULO  gold
Acurácia =  0.7
-----------------------------------------
PROCESSANDO O RÓTULO  white
Acurácia =  0.72
-----------------------------------------
PROCESSANDO O RÓTULO  black
Acurácia =  0.77
-----------------------------------------
PROCESSANDO O RÓTULO  orange
Acurácia =  0.87

Testando k = 4
-----------------------------------------
PROCESSANDO O RÓTULO  red
Acurácia =  0.79
-----------------------------------------
PROCESSANDO O RÓTULO  green
Acurácia =  0.81
-----------------------------------------
PROCESSANDO O RÓTULO  blue
Acurácia =  0.71
-----------------------------------------
PROCESSANDO O RÓTULO  gold
Acurácia =  0.68
--------------------

# Implementação do modelo e testes
### Média geral das acurácias por k

| k   | Acurácia Média |
|-----|----------------|
| 3   | 0.7665 |
| 4   | 0.7533 |
| 5   | 0.7651 |
| 6   | 0.7585 |
| 7   | 0.7717 |
| 8   | 0.7710 |
| 9   | 0.7739 |
| 10  | 0.7680 |
| 11  | 0.7717 |
| 12  | 0.7680 |
| 13  | 0.7725 |
| **14**  | **0.7791** |
| 15  | 0.7769 |
| 16  | 0.7784 |
| 17  | 0.7761 |
| 18  | 0.7688 |
| 19  | 0.7754 |
| 20  | 0.7725 |
  
**Melhor resultado:** k = **14** com acurácia média de **0.7791**.


In [22]:
# Melhor k encontrado
k_otimo = 14

impacto_total = []  # Para guardar impacto de todas as variáveis em todos os labels

# Loop por cada label
for i in range(0, n_labels):
    print(f"\n======================")
    print(f"Analisando variáveis para o rótulo: {labels[i]}")
    print(f"======================")

    # Variáveis preditoras e target
    X_base = flags.drop(columns=labels)
    Y = flags[labels[i]]
    variaveis = X_base.columns

    #1. Acurácia base com todas as variáveis
    mc = pd.DataFrame({'predito_nao':[0,0],
                       'predito_sim':[0,0]},
                       index=['real_nao','real_sim'])

    for j in range(0, n_registros):
        X_treino = X_base.drop([j])
        Y_treino = Y.drop([j])
        X_teste = X_base.iloc[[j], :]
        Y_teste = Y.iloc[j]

        modelo = KNeighborsClassifier(n_neighbors=k_otimo)
        modelo.fit(X_treino, Y_treino)
        pred = modelo.predict(X_teste)

        if Y_teste == 0:
            if pred == 0: mc.iloc[0,0]+=1
            else: mc.iloc[0,1]+=1
        else:
            if pred == 0: mc.iloc[1,0]+=1
            else: mc.iloc[1,1]+=1

    acuracia_base = (mc.iloc[0,0] + mc.iloc[1,1]) / n_registros
    print(f"Acurácia base: {round(acuracia_base, 2)}")

    #2. Testa removendo cada variável
    impacto_label = []
    for var in variaveis:
        mc_var = pd.DataFrame({'predito_nao':[0,0],
                               'predito_sim':[0,0]},
                               index=['real_nao','real_sim'])
        
        X_var = X_base.drop(columns=[var])

        for j in range(0, n_registros):
            X_treino = X_var.drop([j])
            Y_treino = Y.drop([j])
            X_teste = X_var.iloc[[j], :]
            Y_teste = Y.iloc[j]

            modelo = KNeighborsClassifier(n_neighbors=k_otimo)
            modelo.fit(X_treino, Y_treino)
            pred = modelo.predict(X_teste)

            if Y_teste == 0:
                if pred == 0: mc_var.iloc[0,0]+=1
                else: mc_var.iloc[0,1]+=1
            else:
                if pred == 0: mc_var.iloc[1,0]+=1
                else: mc_var.iloc[1,1]+=1

        acuracia_var = (mc_var.iloc[0,0] + mc_var.iloc[1,1]) / n_registros
        perda = acuracia_base - acuracia_var
        impacto_label.append({"label": labels[i], "variavel": var, "impacto": perda})
        impacto_total.append({"label": labels[i], "variavel": var, "impacto": perda})

    # Ranking por label
    df_rank_label = pd.DataFrame(impacto_label).sort_values(by="impacto", ascending=False)
    print("\nRanking de variáveis por impacto neste label:")
    print(df_rank_label)

#Ranking geral
df_rank_geral = pd.DataFrame(impacto_total).groupby("variavel")["impacto"].mean().reset_index()
df_rank_geral = df_rank_geral.sort_values(by="impacto", ascending=False)

print("\n======================")
print("Ranking geral de variáveis por impacto médio:")
print("======================")
print(df_rank_geral)



Analisando variáveis para o rótulo: red
Acurácia base: 0.8299

Ranking de variáveis por impacto neste label:
   label       variavel   impacto
59   red  botright_blue  0.046392
64   red   botright_red  0.041237
52   red   topleft_blue  0.030928
11   red       triangle  0.020619
56   red    topleft_red  0.020619
..   ...            ...       ...
30   red     language_6 -0.005155
16   red     landmass_2 -0.010309
20   red     landmass_6 -0.010309
40   red     religion_5 -0.010309
17   red     landmass_3 -0.015464

[66 rows x 3 columns]

Analisando variáveis para o rótulo: green
Acurácia base: 0.8351

Ranking de variáveis por impacto neste label:
    label        variavel   impacto
11  green        triangle  0.025773
4   green         colours  0.025773
62  green  botright_green  0.025773
34  green     language_10  0.025773
12  green            icon  0.025773
..    ...             ...       ...
24  green          zone_4 -0.005155
19  green      landmass_5 -0.005155
6   green         cross